https://www.kaggle.com/datasets/bharatnatrayn/movies-dataset-for-feature-extracion-prediction 

Context
The data is succesfully scrapped from imdb top netflix movies and tvshows.This dataset need clever programming knowledge for feature extraction also you can build a RECOMMENDATION system either GENRE prediction model

Content
The dataset contain more than 9 columns desrcibe the data pattern .I scrap the data from imdb web site using beautifulsoup.it takes a day to learn i am begginer even to data science but i learned quickly web scrapping with advanced python.Through this process i gained lot and also i suggest the data featuring part of this project takes time

Acknowledgements
Definitely the data collection is possible only with help of online stackoverflow and kaggle data science community

Inspiration
Create genre prediction model?

The data is succesfully scrapped from imdb about netflix movies and tvshows.This dataset need clever programming knowledge for feature extraction also you can build a RECOMMENDATION system either GENRE prediction model
ABOUT DATA

MOVIES
         The movie names are given under the column
YEAR
        The year of movie or tv shows telecast for audience
GENRE
        The dataset contain many genres most valuable for recommendation system
RATING
          The audience thought about movie or tv show given
ONE-LINE
           The short description about movie or tv show for audiences first impression
STARS
            The casting of art is define under this columns which makes talk between audience 
VOTES
            The audience express their view under the columns its useful to identify impact make by art 
RUNTIME
            The duration for runing time of art
GROSS
            It gives the total amount earned in worldwide

In [318]:
import numpy as np
import pandas as pd
import matplotlib as plt
import re

In [319]:
df = pd.read_csv("movies.csv",parse_dates= ["YEAR"])

# df.head() +

In [320]:
df.head()

,MOVIES,YEAR,GENRE,RATING,ONE-LINE,STARS,VOTES,RunTime,Gross
0,Blood Red Sky,(2021),"\r\nAction, Horror, Thriller",6.1,\r\nA woman with a mysterious illness is force...,\r\n Director:\r\nPeter Thorwarth\r\n| \r\n...,"21,062",121.0,NaN
1,Masters of the Universe: Revelation,(2021– ),"\r\nAnimation, Action, Adventure",5.0,\r\nThe war for Eternia begins again in what m...,"\r\n \r\n Stars:\r\nChris Wood, ...","17,870",25.0,NaN
2,The Walking Dead,(2010–2022),"\r\nDrama, Horror, Thriller",8.2,\r\nSheriff Deputy Rick Grimes wakes up from a...,\r\n \r\n Stars:\r\nAndrew Linco...,"885,805",44.0,NaN
3,Rick and Morty,(2013– ),"\r\nAnimation, Adventure, Comedy",9.2,\r\nAn animated series that follows the exploi...,\r\n \r\n Stars:\r\nJustin Roila...,"414,849",23.0,NaN
4,Army of Thieves,(2021),"\r\nAction, Crime, Horror",NaN,"\r\nA prequel, set before the events of Army o...",\r\n Director:\r\nMatthias Schweighöfer\r\n...,NaN,NaN,NaN


In [321]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   MOVIES    9999 non-null   object 
 1   YEAR      9355 non-null   object 
 2   GENRE     9919 non-null   object 
 3   RATING    8179 non-null   float64
 4   ONE-LINE  9999 non-null   object 
 5   STARS     9999 non-null   object 
 6   VOTES     8179 non-null   object 
 7   RunTime   7041 non-null   float64
 8   Gross     460 non-null    object 
dtypes: float64(2), object(7)
memory usage: 703.2+ KB


In [322]:
df.columns

Index(['MOVIES', 'YEAR', 'GENRE', 'RATING', 'ONE-LINE', 'STARS', 'VOTES',
       'RunTime', 'Gross'],
      dtype='object')

In [323]:
miss_val = []

[miss_val.append(i) for i in df.columns if any(df[i].isnull())]

miss_val

['YEAR', 'GENRE', 'RATING', 'VOTES', 'RunTime', 'Gross']

In [324]:
df.shape

(9999, 9)

In [325]:
def first_looking(col):
    print("column name    : ", col)
    print("--------------------------------")
    print("per_of_nulls   : ", "%", round(df[col].isnull().sum()/df.shape[0]*100, 2))
    print("num_of_nulls   : ", df[col].isnull().sum())
    print("num_of_uniques : ", df[col].nunique())
    print(df[col].value_counts(dropna = False))

In [326]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
RATING,8179.0,6.921176,1.220232,1.1,6.2,7.1,7.8,9.9
RunTime,7041.0,68.688539,47.258056,1.0,36.0,60.0,95.0,853.0


In [327]:
df.shape

(9999, 9)

In [328]:
df.isnull().sum()*100/df.shape[0]

MOVIES       0.000000
YEAR         6.440644
GENRE        0.800080
RATING      18.201820
ONE-LINE     0.000000
STARS        0.000000
VOTES       18.201820
RunTime     29.582958
Gross       95.399540
dtype: float64

In [329]:
def show_null(df, limit):
    missing = df.isnull().sum()*100/df.shape[0]
    return missing.loc[lambda x: x > limit]

def percentage_null(serials):
    return serials.isnull().sum()/serials.shape[0]*100

In [330]:
show_null(df, 92)

Gross    95.39954
dtype: float64

In [331]:
def fill_most(df, group_col, col_name):
    '''Fills the missing values with the most existing value (mode) in the relevant column according to single-stage grouping'''
    for group in list(df[group_col].unique()):
        cond = df[group_col]==group
        mode = list(df[cond][col_name].mode())
        if mode != []:
            df.loc[cond, col_name] = df.loc[cond, col_name].fillna(df[cond][col_name].mode()[0])
        else:
            df.loc[cond, col_name] = df.loc[cond, col_name].fillna(df[col_name].mode()[0])
    print("Number of NaN : ",df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

In [332]:
def fill_prop(df, group_col, col_name):
    '''Fills the missing values with "ffill and bfill method" according to single-stage grouping'''
    for group in list(df[group_col].unique()):
        cond = df[group_col]==group
        df.loc[cond, col_name] = df.loc[cond, col_name].fillna(method="ffill").fillna(method="bfill")
    df[col_name] = df[col_name].fillna(method="ffill").fillna(method="bfill")
    print("Number of NaN : ",df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

In [333]:
def fill(df, group_col1, group_col2, col_name, method): # method can be "mode" or "median" or "ffill"
    if method == "mode":
        for group1 in list(df[group_col1].unique()):
            for group2 in list(df[group_col2].unique()):
                cond1 = df[group_col1]==group1
                cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
                mode1 = list(df[cond1][col_name].mode())
                mode2 = list(df[cond2][col_name].mode())
                if mode2 != []:
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond2][col_name].mode()[0])
                elif mode1 != []:
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond1][col_name].mode()[0])
                else:
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[col_name].mode()[0])
                
    elif method == "median":
        for group1 in list(df[group_col1].unique()):
            for group2 in list(df[group_col2].unique()):
                cond1 = df[group_col1]==group1
                cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
                df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond2][col_name].median()).fillna(df[cond1][col_name].median()).fillna(df[col_name].median())
                
    elif method == "ffill":           
        for group1 in list(df[group_col1].unique()):
            for group2 in list(df[group_col2].unique()):
                cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
                df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(method="ffill").fillna(method="bfill")
                
        for group1 in list(df[group_col1].unique()):
            cond1 = df[group_col1]==group1
            df.loc[cond1, col_name] = df.loc[cond1, col_name].fillna(method="ffill").fillna(method="bfill")            
           
        df[col_name] = df[col_name].fillna(method="ffill").fillna(method="bfill")
    
    print("Number of NaN : ",df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

LOOKING FOR COLUMNS ONE BY ONE

YEAR

In [334]:
first_looking("YEAR")

column name    :  YEAR
--------------------------------
per_of_nulls   :  % 6.44
num_of_nulls   :  644
num_of_uniques :  438
(2020– )        892
(2021– )        658
NaN             644
(2020)          639
(2019– )        549
               ... 
(1981–1989)       1
(III) (2016)      1
(1995–2001)       1
(2000–2014)       1
(1985– )          1
Name: YEAR, Length: 439, dtype: int64


In [335]:
df.YEAR.value_counts(dropna = False)


(2020– )        892
(2021– )        658
NaN             644
(2020)          639
(2019– )        549
               ... 
(1981–1989)       1
(III) (2016)      1
(1995–2001)       1
(2000–2014)       1
(1985– )          1
Name: YEAR, Length: 439, dtype: int64

In [336]:
df['YEAR'] = df['YEAR'].str.replace(r'[()]',"")
df['YEAR']

C:\Users\TOSHIBA\AppData\Local\Temp\ipykernel_5104\780251319.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['YEAR'] = df['YEAR'].str.replace(r'[()]',"")


0            2021
1          2021– 
2       2010–2022
3          2013– 
4            2021
          ...    
9994       2021– 
9995       2021– 
9996       2022– 
9997       2021– 
9998       2021– 
Name: YEAR, Length: 9999, dtype: object

In [337]:
df['YEAR'] = df.YEAR.str.strip("III")
df['YEAR']

0            2021
1          2021– 
2       2010–2022
3          2013– 
4            2021
          ...    
9994       2021– 
9995       2021– 
9996       2022– 
9997       2021– 
9998       2021– 
Name: YEAR, Length: 9999, dtype: object

In [338]:
df.YEAR.value_counts(dropna=False)

2020–        892
2021–        658
NaN          644
2020         639
2019–        549
            ... 
1941           1
1992–1995      1
1987–1994      1
2009–2017      1
1985–          1
Name: YEAR, Length: 400, dtype: int64

MOVIES

In [339]:
first_looking("MOVIES")

column name    :  MOVIES
--------------------------------
per_of_nulls   :  % 0.0
num_of_nulls   :  0
num_of_uniques :  6817
 Bleach: Burîchi                         65
 Mighty Little Bheem                     64
 Avatar: The Last Airbender              61
 La Reina de Indias y el Conquistador    60
 Dexter                                  48
                                         ..
Ashby                                     1
Syucheu                                   1
#Realityhigh                              1
Eobiseu                                   1
 Heart of Invictus                        1
Name: MOVIES, Length: 6817, dtype: int64


In [340]:
df[df.MOVIES == '#Realityhigh']

,MOVIES,YEAR,GENRE,RATING,ONE-LINE,STARS,VOTES,RunTime,Gross
2311,#Realityhigh,2017,"\r\nComedy, Drama, Romance",5.2,\r\nHigh-achieving high-school senior Dani Bar...,\r\n Director:\r\nFernando Lebrija\r\n| \r\...,"5,947",99.0,NaN


In [341]:
#df.MOVIES=df[df.MOVIES == '#Realityhigh'].replace('#','')

In [342]:
df.MOVIES.sample(100)

9408                                        Caïd
6593                             Bleach: Burîchi
5646                                      Presos
4804    Larry Charles' Dangerous World of Comedy
2243                                 Gangcheolbi
                          ...                   
321       Rurôni Kenshin: Sai shûshô - The Final
9998                              The Imperfects
9866                                 Gokushufudo
5512        Ugly Duckling Series: Boy's Paradise
8742                          Selena: The Series
Name: MOVIES, Length: 100, dtype: object

GENRE

In [343]:
df.GENRE.value_counts(dropna=False)

\r\nComedy                                      852
\r\nAnimation, Action, Adventure                693
\r\nDrama                                       562
\r\nDocumentary                                 498
\r\nCrime, Drama, Mystery                       336
                                               ... 
\r\nAction, Drama, Western                        1
\r\nComedy, Horror, Talk-Show                     1
\r\nFamily, Music, Romance                        1
\r\nAnimation, Adventure, Horror                  1
\r\nDocumentary, Crime, Thriller                  1
Name: GENRE, Length: 511, dtype: int64

In [344]:
df['GENRE'] = df.GENRE.str.strip("\r\n")
df['GENRE']

0           Action, Horror, Thriller            
1       Animation, Action, Adventure            
2            Drama, Horror, Thriller            
3       Animation, Adventure, Comedy            
4              Action, Crime, Horror            
                          ...                   
9994       Adventure, Drama, Fantasy            
9995    Animation, Action, Adventure            
9996              Documentary, Sport            
9997       Adventure, Drama, Fantasy            
9998       Adventure, Drama, Fantasy            
Name: GENRE, Length: 9999, dtype: object

RATING

In [345]:
df.RATING.value_counts(dropna=False)

NaN    1820
7.2     331
7.6     309
7.5     309
7.4     300
       ... 
2.2       1
1.1       1
2.0       1
2.4       1
1.8       1
Name: RATING, Length: 83, dtype: int64

In [346]:
df["RATING"] = df["RATING"].fillna(method="ffill")
df["RATING"]

0       6.1
1       5.0
2       8.2
3       9.2
4       9.2
       ... 
9994    8.8
9995    8.8
9996    8.8
9997    8.8
9998    8.8
Name: RATING, Length: 9999, dtype: float64

one_line

In [347]:
df['one_line'] = df['ONE-LINE']
df.drop('ONE-LINE', axis=1, inplace=True)

In [348]:
df.one_line.value_counts(dropna=False)

\r\nAdd a Plot\r\n                                                                                                                                                                                                                                   1265
\r\nThe Buddis bounce, spin, glide - and giggle. - through their magical world, learning new things and sharing the joy of friendship.                                                                                                                  8
\r\nWith kindness, curiosity and childlike wonder, five best friends explore their colourful world and find the extraordinary in everyday things.                                                                                                       8
\r\nPlot under wraps.                                                                                                                                                                                                                                   7


In [349]:
df['one_line'] = df.one_line.str.strip()
df['one_line']

0       A woman with a mysterious illness is forced in...
1       The war for Eternia begins again in what may b...
2       Sheriff Deputy Rick Grimes wakes up from a com...
3       An animated series that follows the exploits o...
4       A prequel, set before the events of Army of th...
                              ...                        
9994                                           Add a Plot
9995                                           Add a Plot
9996                                           Add a Plot
9997                                           Add a Plot
9998                                           Add a Plot
Name: one_line, Length: 9999, dtype: object

STARS

In [350]:
df.STARS.value_counts(dropna=False)

\r\n                                                                                                                                                       456
\r\n            \r\n    Stars:\r\nEmmanuel Esparza, \r\nEssined Aponte, \r\nManuel Navarro, \r\nÁlvaro Benet\r\n                                            58
\r\n    Directors:\r\nRajiv Chilaka, \r\nKrishna Mohan Chintapatla\r\n                                                                                      49
\r\n    Director:\r\nOliver Driver\r\n| \r\n    Stars:\r\nRorrie D. Travis, \r\nJasmeet Baduwalia, \r\nJacqueline Scislowski, \r\nAbraham Rodriguez\r\n     21
\r\n            \r\n    Star:\r\nMichael J. Woodard\r\n                                                                                                     19
                                                                                                                                                          ... 
\r\n            \r\n    Stars:\r\nDong-wook Ki

In [351]:
df['STARS'] = df.STARS.str.lstrip("'            '\r\n")
df['STARS'].value_counts()

                                                                                                                                                   456
Stars:\r\nEmmanuel Esparza, \r\nEssined Aponte, \r\nManuel Navarro, \r\nÁlvaro Benet\r\n                                                            58
Directors:\r\nRajiv Chilaka, \r\nKrishna Mohan Chintapatla\r\n                                                                                      49
Director:\r\nOliver Driver\r\n| \r\n    Stars:\r\nRorrie D. Travis, \r\nJasmeet Baduwalia, \r\nJacqueline Scislowski, \r\nAbraham Rodriguez\r\n     21
Star:\r\nMichael J. Woodard\r\n                                                                                                                     19
                                                                                                                                                  ... 
Stars:\r\nDong-wook Kim, \r\nMoon Ga-young, \r\nSeul-gi Kim, \r\nJong-Hoon Yoon\r\n           

In [352]:
df["STARS"]= df["STARS"].replace("\r\n", "", regex=True)
df["STARS"]

0       Director:Peter Thorwarth|     Stars:Peri Baume...
1       Stars:Chris Wood, Sarah Michelle Gellar, Lena ...
2       Stars:Andrew Lincoln, Norman Reedus, Melissa M...
3       Stars:Justin Roiland, Chris Parnell, Spencer G...
4       Director:Matthias Schweighöfer|     Stars:Matt...
                              ...                        
9994    Stars:Morgan Taylor Campbell, Chris Cope, Iñak...
9995                                                     
9996    Director:Orlando von Einsiedel|     Star:Princ...
9997    Director:Jovanka Vuckovic|     Stars:Morgan Ta...
9998    Director:Jovanka Vuckovic|     Stars:Morgan Ta...
Name: STARS, Length: 9999, dtype: object

In [353]:
#res = []
#for sub in df["STARS"]:
#    res.append(sub.replace("\r\n", ""))

#print("List after newline character removal : " + str(res))

In [354]:
df.head()

,MOVIES,YEAR,GENRE,RATING,STARS,VOTES,RunTime,Gross,one_line
0,Blood Red Sky,2021,"Action, Horror, Thriller",6.1,Director:Peter Thorwarth| Stars:Peri Baume...,"21,062",121.0,NaN,A woman with a mysterious illness is forced in...
1,Masters of the Universe: Revelation,2021–,"Animation, Action, Adventure",5.0,"Stars:Chris Wood, Sarah Michelle Gellar, Lena ...","17,870",25.0,NaN,The war for Eternia begins again in what may b...
2,The Walking Dead,2010–2022,"Drama, Horror, Thriller",8.2,"Stars:Andrew Lincoln, Norman Reedus, Melissa M...","885,805",44.0,NaN,Sheriff Deputy Rick Grimes wakes up from a com...
3,Rick and Morty,2013–,"Animation, Adventure, Comedy",9.2,"Stars:Justin Roiland, Chris Parnell, Spencer G...","414,849",23.0,NaN,An animated series that follows the exploits o...
4,Army of Thieves,2021,"Action, Crime, Horror",9.2,Director:Matthias Schweighöfer| Stars:Matt...,NaN,NaN,NaN,"A prequel, set before the events of Army of th..."


In [355]:
df.STARS.value_counts()

                                                                                                                   456
Stars:Emmanuel Esparza, Essined Aponte, Manuel Navarro, Álvaro Benet                                                58
Directors:Rajiv Chilaka, Krishna Mohan Chintapatla                                                                  49
Director:Oliver Driver|     Stars:Rorrie D. Travis, Jasmeet Baduwalia, Jacqueline Scislowski, Abraham Rodriguez     21
Star:Michael J. Woodard                                                                                             19
                                                                                                                  ... 
Stars:Dong-wook Kim, Moon Ga-young, Seul-gi Kim, Jong-Hoon Yoon                                                      1
Stars:Charles Demers, Rebecca Husain, Lili Beaudoin, Ashleigh Ball                                                   1
Stars:John Paul Tremblay, Robb Wells, Mike Smith

VOTES

In [356]:
df.VOTES.value_counts(dropna=False)

NaN       1820
7           35
30          31
26          28
32          28
          ... 
1,368        1
2,437        1
8,271        1
10,883       1
911          1
Name: VOTES, Length: 4130, dtype: int64

In [357]:
df["VOTES"] = df["VOTES"].fillna(method="ffill")
df["VOTES"]

0        21,062
1        17,870
2       885,805
3       414,849
4       414,849
         ...   
9994        433
9995        433
9996        433
9997        433
9998        433
Name: VOTES, Length: 9999, dtype: object

RunTime

In [358]:
df.RunTime.value_counts(dropna=False)

NaN      2958
24.0      354
60.0      299
30.0      219
23.0      183
         ... 
542.0       1
629.0       1
498.0       1
189.0       1
165.0       1
Name: RunTime, Length: 262, dtype: int64

In [359]:
df["RunTime"] = df["RunTime"].fillna(method="ffill")
df["RunTime"]

0       121.0
1        25.0
2        44.0
3        23.0
4        23.0
        ...  
9994     38.0
9995     38.0
9996     38.0
9997     38.0
9998     38.0
Name: RunTime, Length: 9999, dtype: float64

GROSS

In [360]:
df.Gross.value_counts(dropna=False)

NaN        9539
$0.01M       22
$0.02M       16
$0.00M       15
$0.03M       10
           ... 
$44.82M       1
$21.36M       1
$54.76M       1
$77.22M       1
$10.40M       1
Name: Gross, Length: 333, dtype: int64

In [361]:
df.drop('Gross', axis=1, inplace=True)

In [362]:
df

,MOVIES,YEAR,GENRE,RATING,STARS,VOTES,RunTime,one_line
0,Blood Red Sky,2021,"Action, Horror, Thriller",6.1,Director:Peter Thorwarth| Stars:Peri Baume...,"21,062",121.0,A woman with a mysterious illness is forced in...
1,Masters of the Universe: Revelation,2021–,"Animation, Action, Adventure",5.0,"Stars:Chris Wood, Sarah Michelle Gellar, Lena ...","17,870",25.0,The war for Eternia begins again in what may b...
2,The Walking Dead,2010–2022,"Drama, Horror, Thriller",8.2,"Stars:Andrew Lincoln, Norman Reedus, Melissa M...","885,805",44.0,Sheriff Deputy Rick Grimes wakes up from a com...
3,Rick and Morty,2013–,"Animation, Adventure, Comedy",9.2,"Stars:Justin Roiland, Chris Parnell, Spencer G...","414,849",23.0,An animated series that follows the exploits o...
4,Army of Thieves,2021,"Action, Crime, Horror",9.2,Director:Matthias Schweighöfer| Stars:Matt...,"414,849",23.0,"A prequel, set before the events of Army of th..."
...,...,...,...,...,...,...,...,...
9994,The Imperfects,2021–,"Adventure, Drama, Fantasy",8.8,"Stars:Morgan Taylor Campbell, Chris Cope, Iñak...",433,38.0,Add a Plot
9995,Arcane,2021–,"Animation, Action, Adventure",8.8,,433,38.0,Add a Plot
9996,Heart of Invictus,2022–,"Documentary, Sport",8.8,Director:Orlando von Einsiedel| Star:Princ...,433,38.0,Add a Plot
9997,The Imperfects,2021–,"Adventure, Drama, Fantasy",8.8,Director:Jovanka Vuckovic| Stars:Morgan Ta...,433,38.0,Add a Plot


In [363]:
df.to_csv("cleaned_movies.csv", index=False)